In [28]:
import requests
import pandas as pd

#chrome-extension://efaidnbmnnnibpcajpcglclefindmkaj/https://ejscreen.epa.gov/mapper/EJAPIinstructions.pdf
#https://ejscreen.epa.gov/mapper/ejsoefielddesc1.html
#https://www.epa.gov/ejscreen/ejscreen-api
# Load the CSV file
file_path = 'Chicago_Tract_ID.csv'
chicago_tracts_df = pd.read_csv(file_path)

def process_tract_id(tract_id):
    # Remove the dot and convert to an integer
    processed_id = int(tract_id * 100)
    # Convert to string
    processed_str = str(processed_id)
    # Pad with zeros on the right to ensure it's 6 digits long
    processed_str = processed_str.ljust(6, '0')
    return processed_str

base_fips = "17031"

# Apply the processing function to the DataFrame
chicago_tracts_df['Processed'] = chicago_tracts_df.iloc[:, 0].apply(process_tract_id)
chicago_tracts_df['FIPS'] = base_fips + chicago_tracts_df['Processed']

# Convert the list of FIPS codes to a set for efficiency
fips_codes = set(chicago_tracts_df['FIPS'])

# DataFrame to store all results
all_data = pd.DataFrame()

# Initialize counters for tracking API calls
success_count = 0
fail_count = 0

# Limit the loop to 100 API calls
for i, fips_code in enumerate(fips_codes):
    if i >= 90:
        break  # Exit the loop after 100 iterations

    # Modify the areatype to "tract"
    url = f"https://ejscreen.epa.gov/mapper/ejscreenRESTbroker1.aspx?namestr={fips_code}&geometry=&distance=&unit=9035&areatype=tract&areaid={fips_code}&f=json"
    response = requests.get(url)
    
    data = response.json()
    if 'data' in data:
        df = pd.json_normalize(data['data'])
        all_data = pd.concat([all_data, df], ignore_index=True)
        success_count += 1
    else:
        fail_count += 1

# Display the counts and the first few rows of the combined data
print (f"Success: {success_count}, Failures: {fail_count}")
all_data[["main.NUM_HOSPITAL","demographics.P_NHBLACK","demographics.TOTALPOP", "main.NUM_TRI"]]

Success: 75, Failures: 15


,main.NUM_HOSPITAL,demographics.P_NHBLACK,demographics.TOTALPOP,main.NUM_TRI
0,0,89,1652,0
1,0,5,4955,10
2,0,3,7382,0
3,0,7,6632,0
4,10,77,6019,1
...,...,...,...,...
70,0,62,1284,1
71,1,76,5081,0
72,0,66,10528,7
73,0,0,3239,2


In [27]:
all_data.head()
all_data[["main.NUM_HOSPITAL","demographics.P_NHBLACK", "main.NUM_TRI", "extras.RAW_HI_DISABILITYPCT", "extras.RAW_CI_FLOOD"]]
# all_data["extras.RAW_CI_FLOOD"]

,main.NUM_HOSPITAL,demographics.P_NHBLACK,main.NUM_TRI,extras.RAW_HI_DISABILITYPCT,extras.RAW_CI_FLOOD
0,0,89,0,34.3%,4%
1,0,5,10,15.7%,13%
2,0,3,0,9.1%,13%
3,0,7,0,9.4%,2%
4,10,77,1,12.1%,6%
5,0,0,6,9%,14%
6,1,19,0,16.5%,14%
7,1,78,0,24.1%,9%
8,0,0,0,9.5%,2%
9,0,89,0,18.5%,16%
